<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/user_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Navigate to project folder and ensure we're on the 'develop' branch
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Update & checkout develop if repo already exists
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (only needed once per session)
%pip install importnb --quiet

# Add firebase folder to Python path and import its notebook as module
sys.path.append(os.path.join(REPO_DIR, "firebase"))
from importnb import Notebook

with Notebook():
    import FireBase as fb   # now fb.add_user, fb.get_user, etc. are available


In [18]:
def login(username: str, password: str) -> bool:
    # Try direct key lookup first
    status, user = fb.get_user(username)
    if status == 200 and user and user.get("password") == password:
        return True

    # Fallback: scan all users and compare 'username' field
    status, all_users = fb.get_all_users()
    if status != 200 or all_users is None:
        return False

    for u in all_users.values():
        if u.get("username") == username and u.get("password") == password:
            return True

    return False



In [19]:
def get_leaderboard(current_user: str, top_n: int = 5):
    """
    Returns a leaderboard with top N users sorted by score.
    If the current user is not in the top N, also returns their actual rank.

    Args:
        current_user (str): the value of "user_name" of the logged-in user
        top_n (int): number of top users to return

    Returns:
        list of (rank, name, score), and index of current_user in full list
    """
    status, all_users = fb.get_user("")  # get all /users
    if status != 200 or not all_users:
        return [], None

    # Build list of (user_name, score)
    all_scores = []
    for uid, user in all_users.items():
        name = user.get("username", uid)
        score = user.get("score", 0)
        all_scores.append((name, score))

    # Sort by score descending
    all_scores.sort(key=lambda x: x[1], reverse=True)

    # Add ranking index
    ranked = [(i+1, name, score) for i, (name, score) in enumerate(all_scores)]

    # Extract top N
    top_entries = ranked[:top_n]

    # Find current user's rank
    user_entry = next(((rank, name, score) for rank, name, score in ranked if name == current_user), None)

    if user_entry and user_entry not in top_entries:
        top_entries.append(user_entry)  # Add current user as last row

    return top_entries, user_entry[0] if user_entry else None

